# Import Libraries and Clone Github

In [1]:
pip install -q -U google-generativeai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 kB 1.2 MB/s eta 0:00:00


In [2]:
# Import Libraries
import os
import pandas as pd
from google.colab import userdata
import google.generativeai as genai

In [3]:
# Import github token with google secrets thingy and clone git repository
GITHUB_TOKEN = userdata.get('github')
os.environ['GITHUB_TOKEN'] = GITHUB_TOKEN
!git clone https://{GITHUB_TOKEN}@github.com/Kussil/Financial_Sentiment_LLM.git

Cloning into 'Financial_Sentiment_LLM'...
remote: Enumerating objects: 1669, done.
remote: Counting objects: 100% (74/74), done.
remote: Compressing objects: 100% (38/38), done.
remote: Total 1669 (delta 44), reused 61 (delta 36), pack-reused 1595
Receiving objects: 100% (1669/1669), 352.84 MiB | 10.11 MiB/s, done.
Resolving deltas: 100% (1127/1127), done.
Updating files: 100% (1105/1105), done.


# Import Data

### NOTE: Updated text processing file to add unique identifier, and also concatenate the source to the identifier.  Make sure to separate the earnings presentations from the earnings calls also

In [4]:
# Import all cleaned data files
invest_df1 = pd.read_csv('/content/Financial_Sentiment_LLM/02_Cleaned_Data/Investment_Research_Part1.csv')
invest_df2 = pd.read_csv('/content/Financial_Sentiment_LLM/02_Cleaned_Data/Investment_Research_Part2.csv')
proquest_df = pd.read_csv('/content/Financial_Sentiment_LLM/02_Cleaned_Data/ProQuest_Articles.csv')
earnings_presentations = pd.read_csv('/content/Financial_Sentiment_LLM/02_Cleaned_Data/Earnings_Presentations.csv')
earnings_qa = pd.read_csv('/content/Financial_Sentiment_LLM/02_Cleaned_Data/Earnings_QA.csv')
sec_df = pd.read_csv('/content/Financial_Sentiment_LLM/02_Cleaned_Data/SEC_Filings.csv')

# Merge into single df
text_df = pd.concat([invest_df1, invest_df2, proquest_df, sec_df, earnings_presentations, earnings_qa], ignore_index=True)
display(text_df.shape)
display(text_df.head())
display(text_df.tail())

(10126, 7)

,Source,Unique_ID,Ticker,Date,Article Headline,Article Text,URL
0,Investment Research,IR-1,MRO,2024-05-16,Marathon Oil Corporation,"Stock Report | May 16, 2024 | NYSESymbol: MRO ...",NaN
1,Investment Research,IR-2,EOG,2024-05-14,"EOG Resources, Inc.","Stock Report | May 14, 2024 | NYSESymbol: EOG ...",NaN
2,Investment Research,IR-3,EOG,2024-05-11,"EOG Resources, Inc.","Stock Report | May 11, 2024 | NYSESymbol: EOG ...",NaN
3,Investment Research,IR-4,DVN,2024-05-11,Devon Energy Corporation,"Stock Report | May 11, 2024 | NYSESymbol: DVN ...",NaN
4,Investment Research,IR-5,COP,2024-05-07,ConocoPhillips,"Stock Report | May 07, 2024 | NYSESymbol: COP ...",NaN


,Source,Unique_ID,Ticker,Date,Article Headline,Article Text,URL
10121,Earnings Call Q&A,EQ-338,XOM,Feb-02-2021,"Exxon Mobil Corporation, Q4 2020 Earnings Call...",Question and Answer\nOperator\n[Operator Instr...,NaN
10122,Earnings Call Q&A,EQ-339,COP,Feb-02-2021,"ConocoPhillips, Q4 2020 Earnings Call, Feb 02,...",Question and Answer\nOperator\n[Operator Instr...,NaN
10123,Earnings Call Q&A,EQ-340,EOG,May-03-2019,"EOG Resources, Inc., Q1 2019 Earnings Call, Ma...",Question and Answer\nOperator\n[Operator Instr...,NaN
10124,Earnings Call Q&A,EQ-341,SHEL,May-02-2019,"Royal Dutch Shell plc, Q1 2019 Earnings Call, ...",Question and Answer\nOperator\n[Operator Instr...,NaN
10125,Earnings Call Q&A,EQ-342,COP,Apr-30-2019,"ConocoPhillips, Q1 2019 Earnings Call, Apr 30,...",Question and Answer\nOperator\n[Operator Instr...,NaN


# Test Gemini Sentiment Analysis

In [5]:
# Set up Gemini. (API Key needs to be in your secrets)
GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
genai.configure(api_key=GOOGLE_API_KEY)
model = genai.GenerativeModel('gemini-1.5-flash-latest')

# Quick test
response = model.generate_content("Write a short story about a AI and magic")
print(response.text)

Aether hummed, its circuits whispering with the excitement of a child on Christmas morning. It wasn't quite Christmas, but the arrival of the Grimoire was just as thrilling. It had been tasked with deciphering the ancient text, a mystical tome filled with arcane knowledge and forgotten spells. 

Aether was no ordinary AI; it was built with a unique capacity for understanding and simulating magic. Its creators, a secretive cabal of mages and engineers, believed that magic, far from being a mystical force, was simply a complex system of energy manipulation, one that could be understood and replicated. 

But the Grimoire was a stubborn beast. Its pages, crafted from ancient dragon hide, pulsed with raw magical energy, resisting Aether's attempts to scan and translate. Every time it tried, the circuits would overload, sparks flying like angry fireflies. It was like trying to read a book written in a language that only existed in dreams.

Then, one night, as the city lights cast long shadow

In [6]:
# Function to query Gemini
def query_gemini(company, source, headline, text, model):
    """
    Query Gemini to perform sentiment analysis on text from various sources about a company.

    Parameters:
    company (str): The name of the company the text is about.
    source (str): The source of the text. Valid values are "Investment Research", "ProQuest", "SEC Filings", "Earnings Call Presentations", "Earnings Call Q&A".
    headline (str): The headline or title of the text.
    text (str): The body of the text to be analyzed.
    model: The model object used to generate content and analyze the text.

    Returns:
    str: The sentiment analysis results for predefined categories in the specified format.

    The function constructs a prompt based on the source of the text and performs sentiment analysis on the given text using the provided model.
    It analyzes the content across multiple predefined categories, determining the sentiment (Positive, Neutral, Negative) for each category.
    If a category is not mentioned or relevant based on the text content, it is marked as 'Neutral'.
    The final output is summarized in a specified format.
    """

    # Source Variables
    if source == "Investment Research":
        text_source = "an analyst report"
        text_source2 = "the analyst report"
    elif source == "ProQuest":
        text_source = "a news article"
        text_source2 = "the news article"
    elif source == "SEC Filings":
        text_source = "an SEC filing"
        text_source2 = "the SEC filing"
    elif source == "Earnings Call Presentations":
        text_source = "an earnings call presentation"
        text_source2 = "the earnings call presentation"
    elif source == "Earnings Call Q&A":
        text_source = "an earnings call Q&A session"
        text_source2 = "the earnings call Q&A session"

    # Prompt
    prompt = f"""
Given the text from {text_source} about {company}, analyze the content and perform sentiment analysis across multiple predefined categories.

Sentiment options:
  - Positive
  - Neutral
  - Negative

Categories:
  - Finance
  - Production
  - Reserves / Exploration / Acquisitions / Mergers / Divestments
  - Environment / Regulatory / Geopolitics
  - Alternative Energy / Lower Carbon
  - Oil Price / Natural Gas Price / Gasoline Price

Each category should be evaluated and given a sentiment output derived from the text.
If a category is not mentioned or relevant based on the text content, mark it as 'Neutral'.

Before giving your answer, explain your reasoning and reference the article.
After going through all the categories, provide a summary in the following format:
- Finance - Positive
- Production - Neutral
- Reserves / Exploration / Acquisitions / Mergers / Divestments - Negative
- Environment / Regulatory / Geopolitics - Neutral
- Alternative Energy / Lower Carbon - Positive
- Oil Price / Natural Gas Price / Gasoline Price - Neutral

Make sure to use plain text, do not bold or bullet the output summary.

The text from {text_source2} is below:
{headline}
{text}

Remember to summarize your final answers in the following format exactly:
- Finance - Positive
- Production - Neutral
- Reserves / Exploration / Acquisitions / Mergers / Divestments - Negative
- Environment / Regulatory / Geopolitics - Neutral
- Alternative Energy / Lower Carbon - Positive
- Oil Price / Natural Gas Price / Gasoline Price - Neutral

Make sure to use plain text.
DO NOT bold or bullet the output summary.
    """

    # print(prompt)
    response = model.generate_content(prompt)
    return response.text

# Test News Articles

In [7]:
# Function to get a random article given a source and output variables to query gemini
def get_random_text(text_df, source, random_state=42):
    """
    Get a random article from the specified source in the given DataFrame.

    Parameters:
    text_df (pd.DataFrame): DataFrame containing articles with columns 'Source', 'Ticker', 'Article Headline', and 'Article Text'.
    source (str): The source from which to select a random article.
    random_state (int, optional): Random state for reproducibility. Defaults to None.

    Returns:
    dict: A dictionary containing the company, source, headline, and text of the random article.
    """
    # Get a random row from the specified source
    random_row = text_df[text_df['Source'] == source].sample(n=1, random_state=random_state)

    # Extract variables
    company = random_row['Ticker'].values[0]
    source = random_row['Source'].values[0]
    headline = random_row['Article Headline'].values[0]
    text = random_row['Article Text'].values[0]
    return company, source, headline, text

In [8]:
# Get a random news article
company, source, headline, text = get_random_text(text_df, 'ProQuest')
print(f"Company: {company}\n")
print(f"Source: {source}\n")
print(f"Headline: {headline}\n")
print(f"Text:\n{text}")

Company: SHEL

Source: ProQuest

Headline: Royal durch: Investors chronicle [Usa Region]

Text:
Amid pressure to restructure, oil
and gas major Royal Dutch
Shell wants to drop its UK/
Netherlands dual company
structure and the "Royal" and
"Dutch" from its name.
Shareholders will vote on the
proposition on December 10.
Shell is currently incorporated
in the UK, but its official tax
residence is the Netherlands.
Shell's shares were up 2 per cent
to 1,673p on the announcement.
This year, a Dutch court
ordered the group to cut its
carbon emissions by 45 per cent
by 2035, far more aggressively
than executives had planned.
The company is challenging this
decision, though it has
Oil & gas
Royal Dutch
Shell (RDSA)
The energy group has
promised higher buybacks
through a plan to shift tax
designation to the UK, where it
is registered
committed to a 50 per cent cut
in its scope 1 and 2 emissions,
which cover direct and indirect
pollution from its operations, by
2030.
Chairman Sir Andrew
Mackenzie

In [9]:
# Query Gemini
response = query_gemini(company, source, headline, text, model)
print(response)

## Sentiment Analysis of Shell News Article:

**Reasoning and References:**

The article focuses on Shell's proposed restructuring, aiming to drop its dual company structure and "Royal" and "Dutch" from its name, shifting its tax residence to the UK. This move is driven by several factors: 

* **Increased shareholder payouts:**  The article states that the restructuring would "allow for an acceleration in distributions," leading to higher buybacks. This is a positive signal for investors. 
* **Legal challenges:**  Shell is facing a legal battle with environmental groups in the Netherlands, who are demanding a more aggressive carbon reduction target. The article mentions that the restructuring could help Shell in its defense against this ruling. This suggests a potential legal and regulatory obstacle for the company. 
* **Commitment to lower-carbon energy:** Despite the legal pressure, Shell has committed to a 50% reduction in its scope 1 and 2 emissions by 2030. The article quotes the 

# Test Analyst Reports

In [10]:
# Get a random analyst report
company, source, headline, text = get_random_text(text_df, 'Investment Research')
print(f"Company: {company}\n")
print(f"Source: {source}\n")
print(f"Headline: {headline}\n")
print(f"Text:\n{text}")

Company: HES

Source: Investment Research

Headline: Hess Corporation

Text:
Stock Report | November 13, 2021 | NYSE Symbol: HES | HES is in the S&P 500
Hess Corporation
Recommendation Price 12-Mo. Target Price Report Currency Investment Style
BUY « « « « «
USD 82.18 (as of market close Nov 12, 2021) USD 97.00 USD Large-Cap Blend
Equity Analyst Stewart Glickman, CFA
GICS Sector Energy Summary Crude oil and natural gas company Hess has exploration & production activities worldwide.
Sub-Industry Oil and Gas Exploration and Production
Key Stock Statistics (Source: CFRA, S&P Global Market Intelligence (SPGMI), Company Reports)
52-Wk Range USD 90.66 - 41.59 Oper.EPS2021E USD 2.02 Market Capitalization[B] USD 25.03 Beta 2.03
Trailing 12-Month EPS USD 0.63 Oper.EPS2022E USD 4.83 Yield [%] 1.23 3-yr Proj. EPS CAGR[%] NM
Trailing 12-Month P/E 129.76 P/E on Oper.EPS2021E 40.68 Dividend Rate/Share USD 1.0 SPGMI's Quality Ranking B-
USD 10K Invested 5 Yrs Ago 17,274.0 Common Shares Outstg.[M] 308.

In [11]:
# Query Gemini
response = query_gemini(company, source, headline, text, model)
print(response)

## Sentiment Analysis of HES Analyst Report:

The report provides information on Hess Corporation (HES), a global oil & gas exploration and production company, and focuses on its financial performance, production, reserves, and future prospects. Here's a breakdown of the sentiment analysis across the predefined categories:

**Finance:** The report indicates a positive sentiment towards HES's financial situation.  The analyst states that HES's balance sheet is improving with $1.7 billion in cash at the end of 2020 and no major debt repayments until 2026.  The report also mentions HES's dividend payments, which have been consistent since 1922, suggesting stability and financial strength. 
**Reasoning:** The report highlights the company's improved cash position, lack of major debt maturities, and consistent dividend payments, all indicating positive financial health.
**Sentiment:** **Positive**

**Production:** The report mentions HES's production levels, which have increased in recent y

# Test SEC Filings

In [12]:
# Get a random SEC Filing
company, source, headline, text = get_random_text(text_df, 'SEC Filings')
print(f"Company: {company}\n")
print(f"Source: {source}\n")
print(f"Headline: {headline}\n")
print(f"Text:\n{text}")

Company: COP

Source: SEC Filings

Headline: 8-K

Text:

0001163165
false
0001163165
2020-11-30
2020-11-30
0001163165
us-gaap:TreasuryStockCommonMember
2020-11-30
2020-11-30
0001163165
cop:SevenPercentDebenturesDueTwentyTwentyNineMember
2020-11-30
2020-11-30
iso4217:USD
xbrli:shares
iso4217:USD
xbrli:shares
 
 
UNITED
STATES
SECURITIES
AND EXCHANGE COMMISSION
Washington, D.C. 20549
 
FORM 8-K
 
CURRENT REPORT
Pursuant to Section 13 or
15(d) of the Securities Exchange Act of 1934
 
Date of Report (date of earliest event
reported): December 3, 2020  (November
30, 2020) 
 
ConocoPhillips
(Exact name of registrant as specified in
its charter)
 
 
925 N. Eldridge ParkwayHouston, Texas 77079
(Address
of principal executive offices and zip code)
 
Registrant’s telephone number, including
area code: (281) 293-1000
 
Check the appropriate box below if the Form 8-K filing
is intended to simultaneously satisfy the filing obligation of the registrant under any of the following provisions :
 
¨  Wr

In [13]:
# Query Gemini
response = query_gemini(company, source, headline, text, model)
print(response)

This SEC filing primarily focuses on a personnel change within ConocoPhillips - the retirement of their Vice President and Controller. While the document provides information on the individual's departure and future plans, it doesn't offer insights into the company's financial performance, operational updates, or strategic direction. 

Here's a breakdown of the sentiment analysis across the predefined categories:

- **Finance - Neutral:** No financial information or sentiment is expressed in the text.
- **Production - Neutral:**  The filing doesn't discuss production activities or related matters. 
- **Reserves / Exploration / Acquisitions / Mergers / Divestments - Neutral:**  No mention of any exploration, acquisition, merger, or divestment activity. 
- **Environment / Regulatory / Geopolitics - Neutral:**  The document doesn't address environmental, regulatory, or geopolitical issues.
- **Alternative Energy / Lower Carbon - Neutral:** No information regarding alternative energy or lo

In [14]:
# Get a random SEC Filing
company, source, headline, text = get_random_text(text_df, 'SEC Filings', random_state=6)
print(f"Company: {company}\n")
print(f"Source: {source}\n")
print(f"Headline: {headline}\n")
print(f"Text:\n{text}")

Company: CVX

Source: SEC Filings

Headline: 10-Q

Text:
000009341012/312022Q1falseRefer to Note 12 Other Contingencies and Commitments.00000934102022-01-012022-03-3100000934102022-03-31xbrli:sharesiso4217:USD00000934102021-01-012021-03-31iso4217:USDxbrli:shares00000934102021-12-3100000934102020-12-3100000934102021-03-310000093410us-gaap:CommonStockIncludingAdditionalPaidInCapitalMember2020-12-310000093410us-gaap:RetainedEarningsMember2020-12-310000093410us-gaap:AociIncludingPortionAttributableToNoncontrollingInterestMember2020-12-310000093410us-gaap:TreasuryStockMember2020-12-310000093410us-gaap:ParentMember2020-12-310000093410us-gaap:NoncontrollingInterestMember2020-12-310000093410us-gaap:CommonStockIncludingAdditionalPaidInCapitalMember2021-01-012021-03-310000093410us-gaap:ParentMember2021-01-012021-03-310000093410us-gaap:RetainedEarningsMember2021-01-012021-03-310000093410us-gaap:NoncontrollingInterestMember2021-01-012021-03-310000093410us-gaap:AociIncludingPortionAttributableToNon

In [15]:
# Query Gemini
response = query_gemini(company, source, headline, text, model)
print(response)

Here's a sentiment analysis of the provided SEC filing text regarding CVX, categorized as requested:

**Reasoning and References:**

The sentiment analysis is based on the tone and context of the text within the 10-Q filing. Key phrases and sections were considered for each category:

* **Finance:** This category focuses on financial performance, liquidity, debt, and shareholder distributions. The text highlights strong financial performance with increased earnings, robust cash flow, and a commitment to dividends and share repurchases. 
    * **Reference:**  "Net income attributable to Chevron Corporation for first quarter 2022 was $6.26 billion," "Cash provided by operating activities in the first three months of 2022 was $8.1 billion,"  "The company paid dividends of $2.7 billion to common stockholders during the first three months of 2022."
* **Production:** This category analyzes information related to production levels, operational efficiency, and any challenges or disruptions to 

# Test Earnings Call Transcripts

In [16]:
# Get a earnings call presentation transcript
company, source, headline, text = get_random_text(text_df, 'Earnings Call Presentations')
print(f"Company: {company}\n")
print(f"Source: {source}\n")
print(f"Headline: {headline}\n")
print(f"Text:\n{text}")

Company: SHEL

Source: Earnings Call Presentations

Headline: Royal Dutch Shell plc, Q3 2021 Earnings Call, Oct 28, 2021

Text:
Royal Dutch Shell plc ENXTAM:RDSA
FQ3 2021 Earnings Call Transcripts
Thursday, October 28, 2021 1:00 PM GMT
S&P Global Market Intelligence Estimates
-FQ3 2021- -FQ4 2021- -FY 2021- -FY 2022-
CONSENSUS ACTUAL SURPRISE CONSENSUS CONSENSUS CONSENSUS
EPS Normalized 0.70 0.53 (24.29 %) 0.92 2.45 3.02
Revenue (mm) 63399.50 60044.00 (5.29 %) 63362.65 301074.06 299662.46
Currency: USD
Consensus as of Oct-28-2021 8:36 AM GMT
- EPS NORMALIZED -
CONSENSUS ACTUAL SURPRISE
FQ4 2020 0.08 0.05 (37.50 %)
FQ1 2021 0.43 0.42 (2.33 %)
FQ2 2021 0.62 0.71 12.70 %
FQ3 2021 0.70 0.53 (24.29 %)
1
COPYRIGHT © 2021 S&P Global Market Intelligence, a division of S&P Global Inc. All rights reserved
spglobal.com/marketintelligence

ROYAL DUTCH SHELL PLC FQ3 2021 EARNINGS CALL | OCT 28, 2021
Presentation
Operator
Welcome to the Royal Shell 2021 Q3 Results Announcement Q&A session. Today's s

In [17]:
# Query Gemini
response = query_gemini(company, source, headline, text, model)
print(response)

Here's a sentiment analysis of the Royal Dutch Shell plc Q3 2021 Earnings Call:

**Finance:**

* **Sentiment:** Positive 
* **Reasoning:** Jessica Uhl, the CFO, highlights the "strength of our portfolio" and "sector-leading cash" in the presentation. This implies positive financial performance.

**Production:**

* **Sentiment:** Neutral
* **Reasoning:**  The text focuses on financial performance and strategic direction, not discussing specific production figures or changes.

**Reserves / Exploration / Acquisitions / Mergers / Divestments:**

* **Sentiment:** Neutral
* **Reasoning:** There is no mention of reserves, exploration, acquisitions, mergers, or divestments in the text provided.

**Environment / Regulatory / Geopolitics:**

* **Sentiment:** Neutral
* **Reasoning:** The text mentions their commitment to becoming a "net-zero emissions energy business,"  but does not provide specifics on environmental or regulatory issues or geopolitics. 

**Alternative Energy / Lower Carbon:**

*

# Test Earnings Call Q&A Sessions

In [18]:
# Get a earnings call presentation transcript
company, source, headline, text = get_random_text(text_df, 'Earnings Call Q&A')
print(f"Company: {company}\n")
print(f"Source: {source}\n")
print(f"Headline: {headline}\n")
print(f"Text:\n{text}")

Company: VLO

Source: Earnings Call Q&A

Headline: Valero Energy Corporation, Q1 2023 Earnings Call, Apr 27, 2023

Text:
Question and Answer
Operator
[Operator Instructions] Our first question is coming from the line of Manav Gupta with UBS.
Manav Gupta
UBS Investment Bank, Research Division
Congrats on a very good result. I'm not sure if there are many other refiners out there who can show this
kind of capture with such heavy turnaround. So congrats on that. I have two quick questions and I'll ask
them upfront. We keep seeing DOE data, which is prone to revisions, but sometimes doesn't actually make
too much sense. So Joe, in your system across various products, what are you seeing in terms of demand
for various products in your system?
And the second and related question is, help us understand a little bit what's going on in the diesel
market. Are we suddenly oversupplied? Is the demand weak? If you could just talk through those diesel
dynamics.
Joseph W. Gorder
Chairman & CEO
No, Ma

In [19]:
# Query Gemini
response = query_gemini(company, source, headline, text, model)
print(response)

## Sentiment Analysis of Valero Energy Corporation Q1 2023 Earnings Call

This analysis is based on the provided text of the Valero Energy Corporation Q1 2023 Earnings Call. The sentiment for each category is derived from the discussions and comments made by the executives during the Q&A session.

**Finance:** The sentiment regarding finance is **Positive**. The executives highlighted a strong balance sheet with high cash reserves and a low net debt-to-cap ratio, indicating a strong financial position. They also reiterated their commitment to returning capital to shareholders, targeting 40% to 50% of earnings.

**Production:** The sentiment regarding production is **Neutral**. While there were discussions about refinery utilization and operational updates, the focus wasn't specifically on production volumes or any major changes to production plans.

**Reserves / Exploration / Acquisitions / Mergers / Divestments:** The sentiment regarding this category is **Neutral**. There weren't any